# Drive & Environment Setup

In [2]:
IN_COLAB = False

try:
  import google.colab
  IN_COLAB = True
except:
  pass

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi_GDrive
  PROJECT_PATH = '/content/gdrive/MyDrive/Tesi_GDrive'
  #!ls
else:
  %cd /Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive
  PROJECT_PATH = '/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive'

import warnings
import pickle
import sys
import os

if IN_COLAB == True:
  sys.path.insert(0, os.path.abspath('functions'))
  sys.path.insert(0, os.path.abspath(''))
else:
  sys.path.insert(0, os.path.abspath('functions'))
  sys.path.insert(0, os.path.abspath(''))

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from keras.models import Sequential, Model
from keras.layers import Dense, Lambda
from keras.layers import Activation
from keras.activations import exponential, relu
from keras import backend as K
import tensorflow as tf
#from tensorflow.keras.layers import Input, Layer, InputSpec, Reshape
#from tensorflow.keras import initializers
#from tensorflow.keras.optimizers import Adam
tf.config.run_functions_eagerly(True)
from scipy.optimize import minimize, differential_evolution

from pkl import store_data, load_data
#from functions.compressor import compress
from functions.compressor_param import compress_2
from functions.mapping import ind2sub, sub2ind3d

import random
from collections import defaultdict

import time

print("Libraries imported")

/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive


2023-05-25 14:38:10.016622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Libraries imported


# Data Import

In [3]:
cta30_def_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_def.csv')  
df_def = pd.read_csv(cta30_def_path)


cta30_18_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2018.csv')
cta30_19_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2019.csv')
cta30_20_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2020.csv')
cta30_21_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2021.csv')
cta30_22_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2022.csv')

df18= pd.read_csv(cta30_18_path)
df19= pd.read_csv(cta30_19_path)
df20= pd.read_csv(cta30_20_path)
df21= pd.read_csv(cta30_21_path)
df22= pd.read_csv(cta30_22_path)

df = pd.concat([df18, df19, df20, df21, df22], ignore_index=True)

[r,c] = df.shape
print("The dataset has", r, "rows and", c, "columns")

The dataset has 2176378 rows and 76 columns


# Data Preprocessing - "Compression"

## Normalization

In [4]:

scale_norm = MinMaxScaler()
df_norm = pd.DataFrame(scale_norm.fit_transform(df), columns=df.columns)

## Standardization

In [5]:
st_scale = StandardScaler()
df_st = pd.DataFrame(st_scale.fit_transform(df), columns=df.columns)

# Data Preprocessing - "Clustering"

## K-Means

In [9]:
def K_Means_silhouette_best_model(df, k_max):

    X = df.values

    n_clusters_range = range(2, k_max+1)

    silhouettes_score = []
    models = []

    for n_clusters in n_clusters_range:

        kmeans = KMeans(n_clusters=n_clusters)

        labels = kmeans.fit_predict(X)
        silhouette_avg = silhouette_score(X, labels)

        silhouettes_score.append(silhouette_avg)
        models.append(kmeans)
    
    best_model_idx = np.argmax(silhouettes_score)
    best_model = models[best_model_idx]
    best_n_clusters = n_clusters_range[best_model_idx]
    print("Best model has", best_n_clusters, "clusters")

    best_model.fit(X)
    labels = best_model.predict(X)

    df['cluster'] = pd.Series(labels, index=df.index)

    return df, best_model, best_n_clusters



In [10]:
st_kmeans_return = K_Means_silhouette_best_model(df_st, 100)
norm_kmeans_return = K_Means_silhouette_best_model(df_norm, 100)

## K-Medioids

## Agglomerative

## Gaussian Mixture